In [1]:
from scipy.optimize import *
import pandas as pd 
import numpy as np 
from numpy import array
from numpy import mean
from numpy import cov
from numpy.linalg import eig
from tqdm import tqdm
from pulp import *
import plotly.express as px 
from sklearn.decomposition import PCA
import random
import plotly.io as pio 
pio.renderers.default = "browser"
from pandas_datareader import data

In [2]:
    def meanRetAn(data):             
        Result = 1
        
        for i in data:
            Result *= (1+i)
            
        Result = Result**(1/float(len(data)/52))-1
        
        return(Result)

In [3]:
df = pd.read_csv("ETFs_info.csv", index_col="Ticker")
df["Inception"] = pd.to_datetime(df["Inception"])
df = df[df["Inception"] <= "2015-01-01"]

fundamental_df = pd.read_csv("fund_risk_cluster_reduced.csv",index_col="Ticker") #Fundamental Clustering Data
weekly_return = pd.read_csv("WeeklyReturns.csv",index_col="Date") #Weekly Return Data
fundamental_df = fundamental_df.loc[fundamental_df.index.intersection(weekly_return.columns)]
fundamental_df = fundamental_df.loc[fundamental_df.index.intersection(df.index)]
len(fundamental_df)

695

In [4]:
def calculate_pdi(num_assets, tickers, weekly_returns): 
    
    def meanRetAn(data):             
        Result = 1
        
        for i in data:
            Result *= (1+i)
            
        Result = Result**(1/float(len(data)/52))-1
        
        return(Result)

    pca = PCA()
    PDI_dict = {}
    samples = [["SPY"]]
    for number in range(3,num_assets, 1):
        for i in range(1,2000):
            #samples.extend([list(x) for x in combinations(selected_tickers, number_of_assets)])
            samples.append(random.sample(list(tickers),number))
    samples_mini = []
    for i in samples:
        if i not in samples_mini:
            samples_mini.append(i)


    
    for i,y in zip(samples_mini,range(1,len(samples_mini)+1)):
        n_assets = len(i)
        portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
        port_weekly_return = weekly_returns[i].mul(portfolio_weights_ew,axis=1).sum(axis=1)
        ann_ret = meanRetAn(list(port_weekly_return))
        an_cov = weekly_returns[i].cov()
        port_std = np.sqrt(np.dot(portfolio_weights_ew.T, np.dot(an_cov, portfolio_weights_ew)))*np.sqrt(52)
        corr_matrix = np.array(weekly_returns[i].corr())
        principalComponents = pca.fit(corr_matrix)
        PDI = 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1
        PDI_dict[y] = {}
        PDI_dict[y]["PDI_INDEX"] = PDI
        PDI_dict[y]["# of Assets"] = len(i)
        PDI_dict[y]["Assets"] = i
        PDI_dict[y]["Sharpe Ratio"] = ann_ret/port_std
        PDI_dict[y]["Annual Return"] = ann_ret
        PDI_dict[y]["Annual STD"] = port_std
    

        


    PDI_DF = pd.DataFrame(PDI_dict).T
    #PDI_DF["Assets"] = PDI_DF["Assets"].astype(str)
    PDI_DF["# of Assets"] = PDI_DF["# of Assets"].astype(str)
    PDI_DF["Sharpe Ratio"] = PDI_DF["Sharpe Ratio"].astype(float)
    PDI_DF["Annual STD"] = PDI_DF["Annual STD"].astype(float)
    PDI_DF["PDI_INDEX"] = PDI_DF["PDI_INDEX"].astype(float)
    PDI_DF["Annual Return"] = PDI_DF["Annual Return"].astype(float)
    SPY_DF = PDI_DF.iloc[0,:]
    return PDI_DF,SPY_DF

In [5]:
num_as = 12
sel_tick = list(fundamental_df.index)

PDI_DF, SPY_DF = calculate_pdi(num_as, sel_tick, weekly_return)

In [6]:
PDI_DF.head()

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD
1,NaN,1,[SPY],0.818855,0.130413,0.159263
2,1.210099,3,"[KRE, CHIQ, FPA]",0.564758,0.120226,0.212881
3,1.204287,3,"[FYT, SYG, MDYG]",0.563357,0.115168,0.204432
4,1.750332,3,"[MMTM, IAK, ZDEU]",0.523091,0.094329,0.180331
5,1.385366,3,"[FENY, DFJ, PYZ]",0.134866,0.030540,0.226449


In [7]:
fig = px.scatter(PDI_DF, x='PDI_INDEX', y='Sharpe Ratio', color="# of Assets")
fig.show()

In [8]:
fig = px.box(PDI_DF, x="# of Assets", y="Annual STD")
fig.show()

In [9]:
print("min PDI",PDI_DF["PDI_INDEX"].min())
print("max PDI",PDI_DF["PDI_INDEX"].max())

min PDI 1.000041607189078
max PDI 4.787518460819253


In [39]:
PDI_DF

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD
1,NaN,1,[SPY],0.818855,0.130413,0.159263
2,1.210099,3,"[KRE, CHIQ, FPA]",0.564758,0.120226,0.212881
3,1.204287,3,"[FYT, SYG, MDYG]",0.563357,0.115168,0.204432
4,1.750332,3,"[MMTM, IAK, ZDEU]",0.523091,0.094329,0.180331
5,1.385366,3,"[FENY, DFJ, PYZ]",0.134866,0.030540,0.226449
...,...,...,...,...,...,...
17988,2.343309,11,"[DBJP, ZGBR, DBEM, GDX, DVYA, KBWB, VONE, UPRO...",0.651059,0.140238,0.215400
17989,3.421300,11,"[HDV, DOL, CHIQ, EFG, VGT, IQDF, XSW, IYF, DWA...",0.728567,0.128262,0.176047
17990,2.047216,11,"[XSMO, IQDF, KIE, RING, FMAT, FENY, EXI, UWM, ...",0.367182,0.079958,0.217762
17991,2.540120,11,"[EEMO, EWT, SYLD, FRAK, PXF, VBK, EPS, EPI, SC...",0.433353,0.081399,0.187836


In [37]:
a = PDI_DF["PDI_INDEX"].quantile(.5)
a

2.0489481007143064

In [42]:
 df_1 = PDI_DF[PDI_DF["PDI_INDEX"] <= a]
 df_1

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD
2,1.210099,3,"[KRE, CHIQ, FPA]",0.564758,0.120226,0.212881
3,1.204287,3,"[FYT, SYG, MDYG]",0.563357,0.115168,0.204432
4,1.750332,3,"[MMTM, IAK, ZDEU]",0.523091,0.094329,0.180331
5,1.385366,3,"[FENY, DFJ, PYZ]",0.134866,0.030540,0.226449
6,1.134295,3,"[TOLZ, RFG, XLY]",0.537009,0.098377,0.183195
...,...,...,...,...,...,...
17969,1.313271,11,"[SPLV, EWC, QLD, DTD, XLV, EMQQ, EWM, PBE, SAA...",0.572785,0.098076,0.171227
17978,1.119875,11,"[SCHE, EUSA, JKI, DLS, ASEA, XMLV, DOO, SDY, M...",-0.147118,-0.020176,0.137139
17984,1.718354,11,"[SIL, XLG, QQQE, PSJ, FDT, RDVY, CN, SDIV, FNC...",0.684111,0.132154,0.193176
17987,1.192784,11,"[TOK, SMEZ, EZA, BBH, GXF, TNA, UDOW, EWT, PSQ...",0.425618,0.091966,0.216077


In [32]:
PDI_DF["Sharpe Ratio"].quantile([0.5])

0.5    0.497023
Name: Sharpe Ratio, dtype: float64

In [12]:
#0 - 0.25 quantile
df_1 = PDI_DF[(PDI_DF["PDI_INDEX"] <= 2.04)]
df_2 = PDI_DF[(PDI_DF["PDI_INDEX"] > 2.04) ]
# #0.25 - 0.5 quantile
# df_2 = PDI_DF[(PDI_DF["PDI_INDEX"] <= 1.647235)&(PDI_DF["PDI_INDEX"] > 1.181088) & (PDI_DF["Sharpe Ratio"] >= 0)]
# #0-.5 - 0.75 quantile
# df_3 = PDI_DF[(PDI_DF["PDI_INDEX"] > 1.647235)&(PDI_DF["PDI_INDEX"] <= 2.143900)& (PDI_DF["Sharpe Ratio"] >= 0)]
# #0.75-1
# df_4 = PDI_DF[(PDI_DF["PDI_INDEX"] > 2.143900) & (PDI_DF["Sharpe Ratio"] >= 0)]

In [48]:
qq_dict = {}
for i in range(3,11,1):
    qq_dict[i] = {}
    dff_pdi = PDI_DF[PDI_DF["# of Assets"] == str(i)]
    q_mid = dff_pdi["Sharpe Ratio"].quantile(0.5)
    df_1 = dff_pdi[dff_pdi["PDI_INDEX"] < q_mid]
    df_2 = dff_pdi[dff_pdi["PDI_INDEX"] >= q_mid]

    for frame,q in zip([df_1,df_2],["qq","qqq"]):
        qq_dict[i][q] = {}
        for ii in enumerate(frame["Assets"]):
            n_assets = len(ii[1])
            portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
            port_weekly_return = weekly_return[ii[1]].mul(portfolio_weights_ew,axis=1).sum(axis=1)
            qq_dict[i][str(q)][str(ii[0])] = port_weekly_return




In [50]:
qq_dict[5]["qq"].keys()

dict_keys([])

In [165]:
for i in range(3,11,1):
    
    testtest = pd.DataFrame()
    testtest["q1"] = pd.DataFrame(qq_dict["q1"]).mean(axis=1)
    testtest["q2"] = pd.DataFrame(qq_dict["q2"]).mean(axis=1)
    testtest = testtest.cumsum(axis=0)
    fig = px.line(testtest, x= testtest.index, y=testtest.columns)
    fig.show()

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD
2,1.015579,3,"[IEV, DAX, LTL]",0.227715,0.054047,0.237344
3,1.559447,3,"[GDX, FCG, XSW]",0.175919,0.050042,0.284458
4,1.615490,3,"[EIS, ARKG, HACK]",0.790402,0.182831,0.231315
5,1.834083,3,"[DBAW, IYE, SCHD]",0.198582,0.039487,0.198846
6,1.485642,3,"[EQWL, RWJ, ARKQ]",0.743141,0.160735,0.216292
...,...,...,...,...,...,...
1996,1.042407,3,"[EEMV, ACWX, WBIE]",0.371361,0.049966,0.134547
1997,1.274512,3,"[PBS, PFM, UGE]",0.682728,0.138336,0.202622
1998,1.359170,3,"[DES, TECL, XMMO]",0.863799,0.268921,0.311324
1999,1.060498,3,"[SUSA, DWX, EQL]",0.572215,0.089466,0.156350


In [163]:
qq_dict[3]

{'q1': {}, 'q2': {}}

In [ ]:
    testtest = pd.DataFrame()
    testtest["q1"] = pd.DataFrame(qq_dict["q1"]).mean(axis=1)
    testtest["q2"] = pd.DataFrame(qq_dict["q2"]).mean(axis=1)
    testtest = testtest.cumsum(axis=0)
    testtest["id"] = testtest.index
    fff = pd.wide_to_long(testtest,stubnames="q", i="id", j="quantile")
    fff = fff.reset_index().set_index("id")
    print(fff)
    fig = px.line(fff, x= fff.index, y="q")
    fig.show()


In [ ]:
len(df_1)

In [ ]:
weekly_return.index = pd.to_datetime(weekly_return.index)

In [ ]:
qq = []
year = []
porti = []
num_as = []
sr = []
ar = []
std = []
for i in list(weekly_return.index.year.unique()):
    for dfs,q in zip([df_1,df_2], ["q1","q2"]):
        for port in dfs["Assets"]:
            n_assets = len(port)
            portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
            port_weekly_return = weekly_return[weekly_return.index.year == i][port].mul(portfolio_weights_ew,axis=1).mean(axis=1)
            ann_ret = meanRetAn(list(port_weekly_return))
            an_cov = weekly_return[weekly_return.index.year == i][port].cov()
            port_std = np.sqrt(np.dot(portfolio_weights_ew.T, np.dot(an_cov, portfolio_weights_ew)))*np.sqrt(52)
            qq.append(q)
            year.append(i)
            porti.append(port)
            num_as.append(len(port))
            sr.append(ann_ret/port_std)
            ar.append(ann_ret)
            std.append(port_std)


In [ ]:
df = pd.DataFrame()
df["year"] = year
df["q"] = qq
df["aret"] = ar
df["prt"] = porti
df["std"] = std
df["num_p"] = num_as
df["prt"] = df["prt"].astype(str)
df["aret"] = df["aret"].astype(float)

In [ ]:
dff = df.groupby(["year", "q"])[["aret","std"]].mean()
dff = pd.DataFrame(dff).reset_index()

In [ ]:
fig = px.bar(dff, x='year', y='aret', color="q",barmode='group', hover_data = ["std"])
fig.show()

In [ ]:
df_1.head()

In [ ]:
px.histogram(df_2, x="Annual Return")

In [173]:

qq_dict = {}

for i,q in zip([df_1,df_2], ["q1","q2"]):
    qq_dict[q] = {}
    for ii in enumerate(i["Assets"]):
        n_assets = len(ii[1])
        portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
        port_weekly_return = weekly_return[ii[1]].mul(portfolio_weights_ew,axis=1).sum(axis=1)
        qq_dict[q][ii[0]] = port_weekly_return



In [174]:
testtest = pd.DataFrame()
testtest["q1"] = pd.DataFrame(qq_dict["q1"]).mean(axis=1)
testtest["q2"] = pd.DataFrame(qq_dict["q2"]).mean(axis=1)
testtest = testtest.cumsum(axis=0)


In [175]:
testtest

,q1,q2
Date,,
2015-01-14,-0.002192,-0.001945
2015-01-21,0.013071,0.014089
2015-01-28,0.007894,0.008584
2015-02-04,0.024714,0.026921
2015-02-11,0.027821,0.031518
...,...,...
2020-12-02,0.570970,0.631153
2020-12-09,0.585042,0.647037
2020-12-16,0.597654,0.661405


In [176]:
fig = px.line(testtest, x= testtest.index, y=testtest.columns )
fig.show()

In [ ]:
tickers = list(fundamental_df.index)
startDate = "2015-01-01"
endDate = "2021-01-01"
dailyPrices = data.DataReader(tickers, 'yahoo', startDate, endDate)

In [ ]:
dailyPrices = dailyPrices["Adj Close"]

In [ ]:
dailyPrices.to_csv("DailyPrices.csv")

In [ ]:
pricesWed = dailyPrices[dailyPrices.index.weekday==2] 

 weeklyPrices = pricesWed.dropna(axis=1)                 #delete NaN columns
    
# GET WEEKLY RETURNS
weeklyReturns = weeklyPrices.pct_change()                  
weeklyReturns = weeklyReturns.drop(weeklyReturns.index[:1]) 

weeklyReturns.to_csv("WeeklyReturns.csv")